In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K
import tensorflow_hub as hub

import tensorflow.keras.layers 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense,
    Dropout
)

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
main_data = pd.read_csv('/Users/andrewsimon/Desktop/IMDBDataset.csv.zip')
main_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
main_data['sentiment'] = main_data['sentiment'].map({'positive':1, 'negative': 0})
main_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [4]:
train_text = main_data['review'].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = main_data['sentiment'].tolist()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(train_text, train_label, test_size=0.2, random_state=1516)

In [12]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable,
                               name="{}_module".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(
            K.squeeze(K.cast(x, tf.string), axis=1),
            as_dict=True,
            signature='default',
            )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [7]:
tf.compat.v1.disable_eager_execution()
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
input_text = tensorflow.keras.layers.Input(shape=(1,), dtype=tf.string)
embedding = ElmoEmbeddingLayer()(input_text)
dense = tensorflow.keras.layers.Dense(256, activation='relu')(embedding)
pred = tensorflow.keras.layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 dense_2 (Dense)             (None, 256)               262400    
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


In [15]:
pred

<tf.Tensor 'dense_3/Sigmoid:0' shape=(?, 1) dtype=float32>

In [121]:
num_classes = 1
batch_size = 512
epochs = 200
learnRate = 0.001

input_text = tf.keras.Input(shape=(1,), dtype="string", name='input_0')
x = ElmoEmbeddingLayer(trainable=False)(input_text)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dense(num_classes, activation="sigmoid")(x)

model = Model(inputs=[input_text], outputs=x)

model.summary()

# 'binary_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

tf.compat.v1.InteractiveSession()
tf.compat.v1.Session().run(tf.compat.v1.global_variables_initializer())
tf.compat.v1.Session().run(tf.compat.v1.tables_initializer())

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


KeyboardInterrupt: 

In [10]:
tf.compat.v1.InteractiveSession()
tf.compat.v1.Session().run(tf.compat.v1.global_variables_initializer())
tf.compat.v1.Session().run(tf.compat.v1.tables_initializer())

In [14]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5


2023-05-03 22:47:01.039477: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_3/bias/Assign' id:7913 op device:{requested: '', assigned: ''} def:{{{node dense_3/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_3/bias, dense_3/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-03 22:47:01.187335: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at resource_variable_ops.cc:678 : NOT_FOUND: Resource localhost/elmo_embedding_layer_1_module/bilm/char_embed/N10tensorflow3VarE does not exist.
2023-05-03 22:47:01.187367: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at resource_variable_ops.cc:678 : NOT_FOUND: Resource localhost/elmo_embedding_layer_1_module/bilm/char_embed/N10tensorflow3VarE does 

FailedPreconditionError: Could not find variable elmo_embedding_layer_1_module/bilm/CNN/b_cnn_4. This could mean that the variable has been deleted. In TF1, it can also mean the variable is uninitialized. Debug info: container=localhost, status error message=Resource localhost/elmo_embedding_layer_1_module/bilm/CNN/b_cnn_4/N10tensorflow3VarE does not exist.
	 [[{{node elmo_embedding_layer_1/elmo_embedding_layer_1_module_apply_default/bilm/CNN_2/add_4/ReadVariableOp}}]]